In [1]:
import os
import sys
import numpy as np
import time

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b

from estimator.NDIS import BasicNDISEstimator
from analysis.monotonicity_certifier_left import f_value_p, fprime_p, fprime_p_cov
import mpmath as mp

In [2]:
def compare_fprime_versions(
    p: float,
    s: float,
    d: int,
    r: float,
    eps: float,
    dps: int = 80,
    tail_sigma: float = 12.0,
):
    """
    Compare fprime_p (original x-space implementation)
    and fprime_p_cov (u-space change-of-variable implementation).

    Prints:
        - fprime_p value
        - fprime_p_cov value
        - absolute difference
        - relative difference (if denominator nonzero)
    """
    print("Comparing f'(p) with two methods")
    print("-----------------------------------")
    print(f"p={p}, s={s}, d={d}, r={r}, eps={eps}, dps={dps}")

    # Set precision globally for safety
    mp.mp.dps = int(dps)

    # --- compute both values ---
    time_start = time.time()
    fp_x = fprime_p(
        p=p,
        s=s,
        d=d,
        r=r,
        eps=eps,
        workers=1,
        dps=dps,
        tail_sigma=tail_sigma,
    )
    print(f"Time taken for fprime_p: {time.time() - time_start} seconds")

    
    time_start = time.time()
    fp_u = fprime_p_cov(
        p=p,
        s=s,
        d=d,
        r=r,
        eps=eps,
        dps=dps,
    )
    print(f"Time taken for fprime_p_cov: {time.time() - time_start} seconds")
    # --- difference ---
    abs_diff = abs(fp_x - fp_u)
    if fp_x != 0:
        rel_diff = abs_diff / abs(fp_x)
    else:
        rel_diff = mp.mpf('nan')

    # --- print results ---
    print("\nOriginal fprime_p (x-space):")
    print(f"   {fp_x}")

    print("\nfprime_p_cov (u-space):")
    print(f"   {fp_u}")

    print("\nAbsolute difference:")
    print(f"   {abs_diff}")

    print("\nRelative difference:")
    print(f"   {rel_diff}")

    print("-----------------------------------")
    return fp_x, fp_u, abs_diff, rel_diff


In [10]:
d = 15
r = 200
eps = 1
s = 0.001

In [11]:
fp_x, fp_u, abs_diff, rel_diff = compare_fprime_versions(
    p=0.01,
    s=0.001,
    d=d,
    r=r,
    eps=eps,
    dps=80,
)


Comparing f'(p) with two methods
-----------------------------------
p=0.01, s=0.001, d=15, r=200, eps=1, dps=80
Time taken for fprime_p: 0.35442328453063965 seconds
Time taken for fprime_p_cov: 19.647454500198364 seconds

Original fprime_p (x-space):
   0.00000000000000000000000049497655321468226625994515093014622013412170675797972450834039407225972714623963

fprime_p_cov (u-space):
   0.00000000000000000000000049497670067982364715536150239437340807329736830731795197666536813663242290994143

Absolute difference:
   1.4746514138089541635146422718793917566154933822746832497406437269576370180143076e-31

Relative difference:
   0.00000029792348834134882082439794818806671648017568004273047250476782425969477090718591
-----------------------------------
